In [1]:
from __future__ import print_function, division
import numpy as np
from os.path import join, expanduser
import matplotlib.pyplot as plt
import yaml  # for pretty-printing dict
from neuralnilm.metrics import run_metrics, across_all_appliances
import pandas as pd

# sklearn evokes warnings from numpy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/home/jack/workspace/python/Lasagne/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "
/usr/local/lib/python2.7/dist-packages/sklearn/utils/__init__.py:11: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility
  from .murmurhash import murmurhash3_32
/usr/local/lib/python2.7/dist-packages/sklearn/utils/sparsetools/__init__.py:3: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility
  from ._min_spanning_tree import minimum_spanning_tree
/usr/local/lib/python2.7/dist-packages/sklearn/utils/sparsetools/_graph_validation.py:5: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility
  from ._graph_tools import csg

In [11]:
TRAIN_HOUSES = {
    'microwave': (1, 2),
    'fridge': (1, 2, 4),
    'dish washer': (1, 2),
    'kettle': (1, 2, 4),
    'washing machine': (1, 5)
}

TEST_HOUSES = {
    'microwave': (5,),
    'fridge': (5,),
    'dish washer': (5,),
    'kettle': (5,),
    'washing machine': (2,)
}

APPLIANCES = TRAIN_HOUSES.keys()

HOUSES = [1, 2, 3, 4, 5]

METRICS = [
    'f1_score',
    'precision_score',
    'recall_score',
    'accuracy_score',
    'relative_error_in_total_energy',
    'total_energy_correctly_assigned',
    'mean_absolute_error'
]

# ALGORITHMS = ['mean', 'zero', 'co', 'fhmm', 'ae', 'rectangles', 'rnn']

ALGORITHMS = ['co', 'fhmm', 'ae', 'rectangles']


ESTIMATES_PATH = expanduser(
    "~/PhD/experiments/neural_nilm/data_for_BuildSys2015/disag_estimates")
GROUND_TRUTH_PATH = expanduser(
    "~/PhD/experiments/neural_nilm/data_for_BuildSys2015/ground_truth_and_mains")

In [3]:
def load(architecture, building_i, appliance):
    # load estimates
    estimates_fname = "{}_building_{}_estimates_{}.csv".format(
        architecture, building_i, appliance)
    estimates_fname = join(ESTIMATES_PATH, estimates_fname)
    y_pred = np.loadtxt(estimates_fname, delimiter=',')

    # load ground truth
    y_true_fname = "building_{}_{}.csv".format(building_i, appliance.replace(' ', '_'))
    y_true_fname = join(GROUND_TRUTH_PATH, y_true_fname)
    y_true = np.loadtxt(y_true_fname, delimiter=',')

    # load mains
    mains_fname = "building_{}_mains.csv".format(building_i)
    mains_fname = join(GROUND_TRUTH_PATH, mains_fname)
    mains = np.loadtxt(mains_fname, delimiter=',')

    return y_true, y_pred, mains

In [4]:
def plot_all(y_true, y_pred, mains, title=None):
    fig, axes = plt.subplots(nrows=3, sharex=True)
    axes[0].plot(y_pred)
    axes[0].set_title('y_pred')
    axes[1].plot(y_true)
    axes[1].set_title('y_true')
    axes[2].plot(mains)
    axes[2].set_title('mains')
    if title:
        fig.set_title(title)
    plt.show()
    return fig, axes

In [5]:
y_true, y_pred, mains = load('rectangles', 1, 'washing machine')
plot_all(y_true, y_pred, mains)

(<matplotlib.figure.Figure at 0x7fd978be16d0>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x7fd96c0b36d0>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x7fd95d33fe90>], dtype=object))

In [6]:
# Run metrics


In [22]:
def calc_metrics(houses):
    scores = pd.Panel(
        np.NaN,
        items=APPLIANCES,
        major_axis=METRICS,
        minor_axis=ALGORITHMS
    )
    
    for appliance in APPLIANCES:
        houses_for_appliance = houses[appliance]
        for algo in ALGORITHMS:
            house_scores = pd.DataFrame(
                np.NaN, columns=houses_for_appliance, index=METRICS)
            for house_i in houses_for_appliance:
                y_true, y_pred, mains = load(algo, house_i, appliance)
                house_scores_dict = run_metrics(y_true, y_pred, mains)
                house_scores_dict.pop('sum_abs_diff')
                house_scores[house_i] = house_scores_dict
                
    return scores
                    

In [25]:
scores = calc_metrics(TEST_HOUSES)

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/metrics.py:1249: UserWarning: The precision and recall are equal to zero for some labels. fbeta_score is ill defined for those labels [False]. 
  average=average)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/metrics.py:1734: UserWarning: The precision and recall are equal to zero for some labels. fbeta_score is ill defined for those labels [False]. 
  average=average)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/metrics.py:1809: UserWarning: The precision and recall are equal to zero for some labels. fbeta_score is ill defined for those labels [False]. 
  average=average)


IOError: [Errno 2] No such file or directory: '/home/jack/PhD/experiments/neural_nilm/data_for_BuildSys2015/disag_estimates/ae_building_5_estimates_microwave.csv'